In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow import keras
from keras import layers, regularizers, optimizers, losses
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocessing
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from keras.utils import set_random_seed, image_dataset_from_directory
from training import prepare_dataset

In [2]:
SEED = 42
DATASET_PATH = "../../../dataset/"
EPOCHS = 150
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
INPUT_SHAPE = (224, 224, 3, )

# `PYTHONHASHSEED` environment variable
os.environ['PYTHONHASHSEED'] = str(SEED)

# Python built-in random, numpy(+ scikit) and tensorflow seed
set_random_seed(SEED)

In [3]:
# Load the training dataset
print("Loading train dataset...")
train_ds = image_dataset_from_directory(
    directory=os.path.join(DATASET_PATH, "train"),
    label_mode="categorical",
    validation_split=None,
    image_size=(224, 224),
    batch_size=None,
    seed=SEED
    )
print("Train dataset loaded!")
print("Labels in the dataset: ", train_ds.class_names)

# Load the validation dataset
print("Loading validation dataset...")
val_ds = image_dataset_from_directory(
    directory=os.path.join(DATASET_PATH, "valid"),
    label_mode="categorical",
    validation_split=None,
    image_size=(224, 224),
    batch_size=None,
    seed=SEED
    )
print("Validation dataset loaded!")

train_ds = train_ds.shuffle(256, seed=SEED, reshuffle_each_iteration=False).take(int(len(train_ds)*0.25))
val_ds = val_ds.shuffle(128, seed=SEED, reshuffle_each_iteration=False).take(int(len(val_ds)*0.25))

train_ds = prepare_dataset(train_ds, augment=True)
val_ds = prepare_dataset(val_ds)

len(train_ds), len(val_ds)

Loading train dataset...
Found 11220 files belonging to 2 classes.
Train dataset loaded!
Labels in the dataset:  ['savory', 'unsavory']
Loading validation dataset...
Found 600 files belonging to 2 classes.
Validation dataset loaded!


(88, 5)

In [5]:
import keras_tuner as kt

def build_model(hp):
    model = keras.models.Sequential()
    model.add(keras.Input(shape=(INPUT_SHAPE)))
    
    # Search first conv
    model.add(layers.Conv2D(
        filters=hp.Choice('conv_1_filter', values=[32, 64]),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        kernel_regularizer=regularizers.l2(1e-2),
        strides=(1, 1)
    ))
    model.add(layers.MaxPooling2D(pool_size=hp.Choice('pool_1_size', values = [3,5])))
    
    # Choose how many conv layers
    for i in range(hp.Int("num_Convolutional_layers", 1, 2)):
        model.add(
            layers.Conv2D(
                filters=hp.Choice(f"conv_{i}_filters", values=[64, 128, 256]),
                kernel_size=(3, 3),
                activation="relu",
                kernel_regularizer=regularizers.l2(1e-2),
                strides=(1, 1)
            )
        )
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.BatchNormalization())

    # Choose how many classifier
    for i in range(hp.Int("num_FullyConnected_layers", 1, 2)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Choice(f"units_{i}", values=[64, 128, 256]),
                activation="relu",
                kernel_regularizer=regularizers.l2(1e-2)
            )
        )
        if hp.Boolean("dropout"): model.add(layers.Dropout(rate=0.25))
    
    model.add(layers.Dense(2, activation="softmax"))

    # Choose the optimizer
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'adamax'], default = 'adamax')
    optimizer = tf.keras.optimizers.get(hp_optimizer)
    # Choose the learning rate
    optimizer.learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-2], default = 1e-3)
                                        
    model.compile(optimizer=optimizer, 
                    loss="categorical_crossentropy", 
                    metrics = ["accuracy"])

    return model

In [6]:
os.makedirs("./keras_tuner/", exist_ok=True)
tuner = kt.BayesianOptimization(build_model,
                                objective=kt.Objective('val_loss', direction="min"),
                                directory='./keras_tuner',
                                max_trials = 20, overwrite=False,
                                project_name='tuned_model')

In [7]:
# The combination of all parameters
tuner.search_space_summary()

Search space summary
Default search space size: 10
conv_1_filter (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64], 'ordered': True}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
pool_1_size (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
num_Convolutional_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': None}
conv_0_filters (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 256], 'ordered': True}
num_FullyConnected_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': None}
units_0 (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 256], 'ordered': True}
dropout (Boolean)
{'default': False, 'conditions': []}
optimizer (Choice)
{'default': 'adamax', 'conditions': [], 'values': ['adam', 'rmsprop', 'adamax'], 'ordered': False}
learning_rate (Choice)
{'default': 0.001, 'conditi

In [8]:
early_stopping = EarlyStopping(monitor="val_loss", patience=12, min_delta=0.0001, restore_best_weights=True, verbose=1)
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, verbose=1)
callbacks = [early_stopping, lr_scheduler]
# Search best hyperparameter
tuner.search(train_ds, epochs=150, validation_data=val_ds, shuffle=True, callbacks=callbacks)

Trial 10 Complete [00h 00m 22s]
val_loss: 0.6964443325996399

Best val_loss So Far: 0.5550560355186462
Total elapsed time: 00h 08m 41s


ValueError: `x0` violates bound constraints.

In [9]:
# Get the top model
models = tuner.get_best_models(num_models=1)
best_model = models[0]
# Build the model.
# best_model.build(input_shape=INPUT_SHAPE)
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 32)      2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 44, 44, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 42, 42, 256)       73984     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 21, 21, 256)      0         
 2D)                                                             
                                                                 
 global_average_pooling2d (G  (None, 256)              0         
 lobalAveragePooling2D)                                          
                                                        

In [10]:
tuner.results_summary()

Results summary
Results in ./keras_tuner\tuned_model
Showing 10 best trials
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 5
pool_1_size: 5
num_Convolutional_layers: 1
conv_0_filters: 256
num_FullyConnected_layers: 1
units_0: 64
dropout: False
optimizer: adam
learning_rate: 0.001
conv_1_filters: 128
units_1: 128
Score: 0.5550560355186462
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 5
pool_1_size: 5
num_Convolutional_layers: 1
conv_0_filters: 256
num_FullyConnected_layers: 1
units_0: 64
dropout: False
optimizer: rmsprop
learning_rate: 0.001
conv_1_filters: 64
units_1: 64
Score: 0.6116705536842346
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 5
pool_1_size: 5
num_Convolutional_layers: 1
conv_0_filters: 256
num_FullyConnected_layers: 1
units_0: 64
dropout: True
optimizer: adam
learning_rate: 0.001
conv_1_filters: 64
units_1: 64
Score: 0.6132145524024963
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 5
pool_1_size: 5
nu

In [11]:
best_model.save("./keras_tuner/" + 'best_hyperparameter_tuned_model')

INFO:tensorflow:Assets written to: ./keras_tuner/best_hyperparameter_tuned_model\assets


INFO:tensorflow:Assets written to: ./keras_tuner/best_hyperparameter_tuned_model\assets
